In [2]:
import pickle
import numpy as np
import pandas as pd

In [3]:
with open("C:\\FRR40\\rat25-main\\temp\\frr40_detections_0.p", 'rb') as f:
    input_features = pickle.load(f)

with open("C:\\FRR40\\rat25-main\\temp\\frr40_objects_0.p", 'rb') as f:
    labels = pickle.load(f)

with open("C:\\FRR40\\rat25-main\\temp\\egoMotionDynamicData_0.p", 'rb') as f:
    ego_features = pickle.load(f)

C:\Users\Q674749\AppData\Local\Temp\ipykernel_6132\4257435610.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  input_features = pickle.load(f)
C:\Users\Q674749\AppData\Local\Temp\ipykernel_6132\4257435610.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to 

In [11]:
feature_columns = ["rcs", "distance", "angleAzimuth", "angleElevation", "radialVelocity", "radialVelocityDomainMax", 'yawrate', 'egospeed' ]
label_columns = ["orientation", "x", "y", "width_edge_mean", "length_edge_mean", "header.origin.x", "header.origin.y", "header.origin.z", "header.origin.roll","header.origin.pitch", "header.origin.yaw"  ]




In [12]:


def preprocess_labels(labels, label_columns):
    labels = labels[label_columns]

    for col in label_columns:
        labels[col] = labels[col].apply(convert_to_numeric)

    return labels

def convert_to_numeric(value):

    if isinstance(value, list):
        return [float(v) if isinstance(v, (int,float,str)) and v != '' else 0 for v in value]
    
    elif isinstance(value, (int, float)):
        return float(value)
    elif isinstance(value, str):
        try:
            return float(value)
        except ValueError:
            return 0
    elif pd.isna(value):
        return 0
    else:
        return 0
    
labels_df = preprocess_labels(labels, label_columns)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
def transform_to_scs(labels):
    
    x_scs_all, y_scs_all, z_scs_all = [], [], []


    for i, row in labels.iterrows():
        roll = row["header.origin.roll"]
        pitch = row["header.origin.pitch"]
        yaw = row["header.origin.yaw"]

        x_header, y_header, z_header = row["header.origin.x"], row["header.origin.y"], row.get("header.origin.z", 0)

        # Rotation offsets
        R_x = np.array([
            [1, 0, 0],
            [0, np.cos(roll), -np.sin(roll)],
            [0, np.sin(roll), np.cos(roll)]
        ])
        R_y = np.array([
            [np.cos(pitch), 0, np.sin(pitch)],
            [0, 1, 0],
            [-np.sin(pitch), np.cos(pitch)]
        ])
        R_z = np.array([
            [np.cos(yaw), -np.sin(yaw), 0],
            [np.sin(yaw), np.cos(yaw), 0],
            [0, 0, 1]
        ])

        # combined rotation matrxi
        R = R_z @ R_y @ R_x

        if len(row["x"]) != len(row["y"]) != len(row.get("z", [0] * len(row["x"]))):
            print(f" row {i} has mismatched lengths:")

        # trasnfrom each object coordinates

        x_scs, y_scs, z_scs = [], [], []
        for x_vcs, y_vcs, z_vcs in zip(row["x"], row["y"], row.get("z", [0] * len(row["x"]))):

            x_vcs_translated = x_vcs - x_header
            y_vcs_translated = y_vcs - y_header
            z_vcs_translated = z_vcs - z_header

            # apply rotation

            vcs_coords = np.array([x_vcs_translated, y_vcs_translated, z_vcs_translated])
            scs_coords = R @ vcs_coords


            # store new coords

            x_scs.append(scs_coords[0])
            y_scs.append(scs_coords[1])
            z_scs.append(scs_coords[2])
        
        # adding trasnformed lists to result lists
        x_scs_all.append(scs_coords[0])
        y_scs_all.append(scs_coords[1])
        z_scs_all.append(scs_coords[2])

    # adding new cols to the labels

    labels["x_scs"] = x_scs_all
    labels["y_scs"] = y_scs_all
    labels["z_scs"] = z_scs_all


    return labels



labels_transformed_scs = transform_to_scs(labels)





ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [ ]:
def find_egosignal(timestamp):
    ind = ego_motion.iloc[(ego_motion['timestamp'] -
                           timestamp).abs().argsort()[:1]].reset_index()
    #print(list(ind))
 
    return ind['egoRotationRates.yawRateVehicleBody.value'].values[0], ind['egoVelocity.egoSpeedCog.egoSpeedCog'].values[0]
ego_motion = pd.read_pickle(
    'C:/Pouyas/Repos/rat25/temp/egoMotionDynamicData_0.p')
detectionlist = pd.read_pickle('C:/Pouyas/Repos/rat25/temp/srr_objects_ipc_0.p')
 
detectionlistlist['yawrate'], objlist['egospeed'] = zip(
    *detectionlistlist['timestamp'].map(find_egosignal))